In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
import os
import matplotlib.pyplot as plt

import pathlib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Definindo o caminho do sistema onde os arquivos que serão analisados estão localizadas

In [43]:
import os.path
import pathlib

path = '../input/simpsons-images/'
train_dir = os.path.join(path,'train') 
test_dir= os.path.join(path,'test') 
data_dir = pathlib.Path(path)
os.path.exists(path)

Visualizando uma imagem da subpasta lisa_simpson

In [44]:
lisa = list(data_dir.glob('test/lisa_simpson/*.jpg'))
PIL.Image.open(str(lisa[0]))

Visualizando uma imagem da subpasta bart_simpson

In [45]:
bart = list(data_dir.glob('test/bart_simpson/*.jpg'))
PIL.Image.open(str(bart[0]))

Convertendo a imagem no modo L para termos uma imagem de canal único e o cmap = 'cinza' converte as cores

In [46]:
bart = list(data_dir.glob('test/bart_simpson/*.jpg'))
image = PIL.Image.open(bart[0]).convert("L")
plt.imshow(image, cmap='gray')

Definindo o tamanho do lote de imagens que será usado e o tamanho das imagens

In [47]:
batch_size = 32
img_height = 180
img_width = 180

Separando o que será usado para treinamento e o que será usao para teste

In [48]:
from torchvision import datasets
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import  SubsetRandomSampler

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)






In [49]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  validation_split=0.5,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Exibindo os nomes das classes

In [50]:
class_names = train_ds.class_names
print(class_names)

Exibindo uma amostra dos dados de trainamento

In [51]:

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Visualizando a barra de cores de algumas imagens

In [52]:

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.colorbar()
    plt.grid(True)
    plt.axis("off")

Exibindo o tamanho do lote de imagem, o tamanho das imagens e os canais de cores RGB do tensor e as informações das labels

In [53]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [54]:
for image_batch, labels_batch in val_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

Dataset.cache mantém as imagens na memória depois de serem carregadas fora do disco durante a primeira época. Isso garantirá que o conjunto de dados não se torne um gargalo ao treinar o modelo
Dataset.prefetch sobrepõe o pré-processamento de dados e a execução do modelo durante o treinamento.

In [55]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Padronizando os valores do canal RGB para estarem no intervalo [0, 1] usando tf.keras.layers.Rescaling

In [56]:
normalization_layer = layers.Rescaling(1./255)

In [57]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

Criando o modelo que possui
três blocos de convolução ( tf.keras.layers.Conv2D )
uma camada de agrupamento máximo ( tf.keras.layers.MaxPooling2D ) em cada bloco
e uma camada totalmente conectada ( tf.keras.layers.Dense ) com 128 unidades em cima dela que é ativada por uma função de ativação ReLU ( 'relu' )

In [58]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

Compilando o modelo

In [59]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Resumo do modelo

In [60]:
model.summary()

Treinando o modelo

In [61]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Gráfico de perda e precisão nos conjuntos de treinamento e validação

In [62]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Mudando o posicionamento da imagem para gerar "mais imagens" para expor o modelo a dados mais generalizados

In [63]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

Visualizando as imagens geradas anteriormente

In [64]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

Adicionando o dropout que descarta aleatoriamente um número de unidades de saída da camada durante o processo de treinamento. Quanto mais unidades desistem, mais forte é a regularização. Isso é análogo ao treinamento da rede para emular um conjunto exponencialmente grande de redes menores

In [65]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

Compilando o modelo

In [66]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Resumo do modelo

In [67]:
model.summary()

Treinando o modelo

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Visualizando os resultados do treinamento

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()